# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [49]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [20]:
path = '../output_data/cities.csv'

cities = pd.read_csv(path)

cities.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Port Alfred,-33.5906,26.8910,70.41,77,13,24.87,ZA,1611605802
1,San Miguel,13.4833,-88.1833,98.60,36,0,9.22,SV,1611605802
2,Vaini,-21.2000,-175.2000,80.60,89,20,6.91,TO,1611605803
3,Ushuaia,-54.8000,-68.3000,53.60,40,40,9.22,AR,1611605803
4,Port-Gentil,-0.7193,8.7815,80.60,83,40,4.61,GA,1611605803


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [21]:
locations = cities[['Lat', 'Lng']]
weights = cities['Humidity'].astype(float)
# Layout
figure_layout = {
     'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weights)
# Add layer
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [22]:
ideal_vacay = cities.loc[((cities['Max Temp'] <=75) & (cities ['Max Temp'] >=55)) & (cities['Humidity'] <=55) & 
                        (cities['Wind Speed'] <=20)]
ideal_vacay.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,Māngrol,21.1167,70.1167,63.82,29,0,13.58,IN,1611605803
20,Kumhāri,21.2667,81.5167,63.34,50,0,4.92,IN,1611605805
25,Constitución,-35.3333,-72.4167,69.26,40,0,16.11,CL,1611605806
57,Tezu,27.9167,96.1667,58.50,41,15,4.05,IN,1611605810
82,Kutum,14.2000,24.6667,63.32,25,85,10.94,SD,1611605814


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [55]:
ideal = ideal_vacay.copy()
ideal['Hotel Name'] = ""
ideal.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
6,Māngrol,21.1167,70.1167,63.82,29,0,13.58,IN,1611605803,
20,Kumhāri,21.2667,81.5167,63.34,50,0,4.92,IN,1611605805,
25,Constitución,-35.3333,-72.4167,69.26,40,0,16.11,CL,1611605806,
57,Tezu,27.9167,96.1667,58.50,41,15,4.05,IN,1611605810,
82,Kutum,14.2000,24.6667,63.32,25,85,10.94,SD,1611605814,


In [56]:
hotels = []

target_search = 'Hotel'
target_coordinates = list(zip(ideal.Lat, ideal.Lng))
target_radius = 5000
target_type = 'hotel'

params = {
    'location': target_coordinates,
    'keyword': target_search,
    'radius': target_radius,
    'type': target_type,
    'key': g_key
}

base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for city in ideal:
    
    response = requests.get(base_url, params=params).json()
    try:
        hotel.append(response['name'])
    except:
        print('not found')
        

# pprint(json.dumps(ideal))

not found
not found
not found
not found
not found
not found
not found
not found
not found
not found


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
